In [43]:
import numpy as np
from collections import Counter

In [44]:
class Node:
    def __init__(self,feature=None,threshold=None,left=None,right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
    
    def _is_leaf_node(self):
        return self.value is not None

In [45]:
class DecisionTree:
    def __init__(self,min_samples_split=2,max_depth=100,n_features=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None
    
    def fit(self,X,y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X,y,0)
    
    def _grow_tree(self,X,y,depth=0):
        n_samples, n_features = X.shape
        labels = np.unique(y)
        self.n_labels = len(labels)
        
        
        #stopping criteria
        if(self.max_depth<= depth or self.min_samples_split > n_samples or self.n_labels == 1):
            value = self._most_common_label(y)
            return Node(value=value)
        
        feature_idxs = np.random.choice(n_features,self.n_features,replace=False)
        
        best_feature,best_threshold = self._best_split(X,y,feature_idxs)
        
        left_idxs,right_idxs = self._split(X[:,best_feature],best_threshold)
        
        left_node = self._grow_tree(X[left_idxs,:],y[left_idxs],depth+1)
        right_node = self._grow_tree(X[right_idxs,:],y[right_idxs],depth+1)
        
        return Node(feature=best_feature,threshold=best_threshold,left=left_node,right=right_node)
    
        
        
        
    def _best_split(self,X,y,feat_idxs):
        
        best_gain = -1
        best_feature,best_threshold = None,None
        for idx in feat_idxs:
            X_c = X[:,feat_idxs]
            
            thresholds = np.unique(X_c)
            
            for thr in thresholds:
                info_gain = self._info_gain(X_c,y,thr)
                
                if best_gain < info_gain:
                    best_gain = info_gain
                    best_feature = idx
                    best_threshold = thr
        
        return best_feature,best_threshold
    
    def _info_gain(self,X_c,y,thr):
        parent_entropy = self._entropy(y)
        
        left_idxs, right_idxs = self._split(X_c,thr)
        
        if (len(left_idxs) == 0 or len(right_idxs) == 0):
            return 0
        
        n_l,n_r = len(left_idxs),len(right_idxs)
        e_l,e_r = self._entropy(y[left_idxs]),self._entropy(y[right_idxs])
        
        children_entropy = ((n_l/len(y)) * e_l ) + ((n_r/len(y)) * e_r)
        
        return parent_entropy - children_entropy
        
    
    def _entropy(self,y):
        hist = np.bincount(y)
        
        px = hist/len(y)
        
        return -np.sum([np.log(p) * p for p in px if p>0])
        
    
    def _split(self,X_c,thr):
        left_idxs = np.argwhere(X_c <= thr).flatten()
        right_idxs = np.argwhere(X_c > thr).flatten()
        
        return left_idxs,right_idxs
        
                    
    
    def _most_common_label(self,y):
        counter = Counter(y)
        #most common label(and the first one in case 2 labels have same number of elements)
        value = counter.most_common(1)[0][0]
        return value
        
    def predict(self,X):
        return np.array([self._traverse_tree(x,self.root) for x in X])
    
    def _traverse_tree(self,x,node):
        if node._is_leaf_node():
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x,node.left)
        
        return self._traverse_tree(x,node.right)
    

In [46]:
import sklearn
from sklearn import datasets

bc = datasets.load_breast_cancer()

X,y = bc.data,bc.target

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [47]:
clf = DecisionTree()
clf.fit(X_train,y_train)

KeyboardInterrupt: 

In [ ]:
preds = clf.predict(X_test)

In [ ]:
acc = np.sum(y_test == preds)/len(y_test)
acc